# Statystyka matematyczna - ćwiczenia laboratoryjne 2023/2024

Ten notatnik zalicza się do grupy zestawów zadań, na podstawie których odbywa się zaliczenie ćwiczeń i podlega zwrotowi do oceny w ustalonym na zajęciach terminie.

Uwagi i wytyczne ogólne dotyczące uzupełniania notatnika:
- Podczas wykonywania zadań należy korzystać wyłącznie z pakietów zaimportowanych na początku notatnika oraz z pakietów wchodzących w skład standardowej biblioteki Pythona, które można zaimportować samodzielnie we wskazanej komórce notatnika.
- Swoje rozwiązania należy wprowadzać wyłącznie w miejce następujących fragmentów kodu:<br/> `# YOUR CODE HERE`<br/> `raise NotImplementedError()`<br/> 
a odpowiedzi tekstowe w komórkach oznaczonych hasłem:<br/> 
`YOUR ANSWER HERE`<br/> 
Nie należy w żaden sposób modyfikować pozostałych fragmentów kodu oraz innych elementów notatnika, w szczególności dodawać lub usuwać komórek oraz zmieniać nazwy pliku.
- Otrzymywane wyniki i odpowiedzi mają być rezultatem wykonania napisanego kodu.
- Zadanie należy wykonać w taki sposób, aby podczas wykonywania kodu nie zostały wyświetlone żadne ostrzeżenia.
- Zawarte w notatniku automatyczne testy mają charakter poglądowy. Dotyczą one wybranych aspektów zadań i mają za zadanie wyłapać podstawowe błędy. Przejście wszystkich testów nie oznacza, że zadanie jest wykonane w całości poprawnie.

Uwagi i wytyczne ogólne dotyczące wizualizacji wyników:
- Wszystkie wykresy powinny być wykonane w jednolitym, przejrzystym i czytelnym stylu, posiadać odpowiednio dobrane proporcje i zakresy wartości osi.
- Wykresy oraz ich osie powinny mieć nadane tytuły. Jeżeli w obrębie figury znajduje się więcej niż jeden wykres to figura również powinna mieć nadany tytuł. 
- Figury powinny mieć ustawione białe tło, tak, aby niezależnie od ustawień notatnika wszystkie elementy wykresów były dobrze widoczne (domyślnie tło jest przeźroczyste co może powodować problemy w notatnikach z ustawionym ciemnym tłem).
- Rozmiar poziomy figur nie powinien przekraczać 20 cali.

Przed odesłaniem zestawu zadań do oceny proszę uzupełnić komórkę z danymi autorów rozwiązania (nazwa zespołu oraz imiona, nazwiska i numery indeksów członków zespołu) oraz upewnić się, że notatnik działa zgodnie z oczekiwaniami. W tym celu należy skorzystać z opcji **Restart Kernel and Run All Cells...** dostępnej na górnej belce notatnika pod symbolem $\blacktriangleright\blacktriangleright$.

Nazwa zespołu:
Członkowie:

---

# Zestaw zadań 8: Estymacja metodą bootstrap

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Miejsce do importu pakietów wchodzących w skład standardowej biblioteki Pythona oraz ustawienie opcji wykorzystywanych pakietów

### Dane do zadań

W celu wygenerowania danych wykorzystywanych w zawartych w notatniku zadaniach i komórkach testowych wykonaj obie poniższe komórki.

In [2]:
sample_1 = pd.DataFrame(data=sp.stats.norm.rvs(loc=5, scale=0.2, size=20, random_state=7), columns=["X"])
sample_1.head()

X
0  5.338105
1  4.906813
2  5.006564
3  5.081503
4  4.842215

In [3]:
x = sp.stats.uniform.rvs(loc=-2, scale=10, size=25, random_state=34)
y = 2*x - 5 + sp.stats.norm.rvs(loc=0, scale=2, size=25, random_state=13)
sample_2 = pd.DataFrame(data=np.array([x, y]).T, columns=["X", "Y"])
sample_2.head()

X         Y
0 -1.614383 -9.653548
1  5.801005  8.109542
2 -1.072962 -7.234931
3  4.328927  4.561478
4 -1.861092 -6.031980

### Zadanie 1: Estymacja bootstapowa parametrów próby losowej [8 pkt]

Przygotuj funkcję `bootstrap_parameter_estimation()`, która korzystając z metody bootstrap będzie dokonywała estymacji przedziałowej podstawowych parametrów próby losowej - wartości oczekiwanej, wariancji i odchylenia standardowego dla danych wejściowych w postaci szeregu szczegółowego zgodnie ze schematem z załączonego do notatnika zestawu wzorów.

W obliczeniach skorzystaj z nieobciążonych estymatorów wariancji i odchylenia standardowego.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`P należy do przedziału [X, Y] przy założeniu poziomu ufności 1-𝛼 = Z`

gdzie P (symbol lub nazwa estymowanego parametru), X, Y i Z są automatycznie uzupełniane przez funkcję, a X i Y dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

Uwagi do wykonania zadania:
 - Automatyczne testy zakładają losowanie prób wg następującego schematu - kolejne próby są losowane w pęli for, a funkcja losująca próbę (np. `pd.sample()`) w kolejnych losowaniach ma ustawioną wartość parametru odpowiadającego za ziarno generatora liczb losowych na wartość `random_state`, `random_state+1`, `random_state+2` itd. 
 - Celem zadania jest napisanie funkcji od podstaw, w rozwiązaniu nie należy korzystać z gotowych funkcji służących do estymacji bootstrapowej.

In [8]:
import numpy as np
def bootstrap_parameter_estimation(data, parameter, number_of_samples, alpha, random_state):
    n = len(data)
    estimates = []
    def calculate_estimates(sample):
        if parameter == "mean":
            return np.mean(sample)
        elif parameter == "var":
            return np.var(sample, ddof=1) # liczba stopni swobody ddof
        elif parameter == "std":
            return np.std(sample, ddof=1)
        else:
            raise ValueError("Nieprawidłowy parametr. Dostępne opcje: 'mean', 'variance', 'std'")
    for _ in range(number_of_samples):
        #np.random.seed(random_state)
        bootstrap_sample = data.sample(n, replace=True, random_state = random_state)
        random_state += 1 
        estimates.append(calculate_estimates(bootstrap_sample))
    lower_bound = np.quantile(estimates, alpha/2)  # np.quantile
    upper_bound = np.quantile(estimates, 1-alpha/2) # 
    print(f"{parameter.capitalize()} należy do przedziału [{lower_bound:.4f}, {upper_bound:.4f}] przy założeniu poziomu ufności 1-{alpha} = {1-alpha}")
    return lower_bound, upper_bound

In [9]:
# Komórka testowa
assert np.all(np.isclose(bootstrap_parameter_estimation(sample_1, "mean", 1000, 0.05, 34), (4.913578501079754, 5.058598650238277)))
assert np.all(np.isclose(bootstrap_parameter_estimation(sample_1, "var", 2000, 0.02, 31), (0.009780029754528753, 0.05012098474121563)))
assert np.all(np.isclose(bootstrap_parameter_estimation(sample_1, "std", 1500, 0.1, 39), (0.11848597559111698, 0.20690074715252046)))

Mean należy do przedziału [4.9136, 5.0586] przy założeniu poziomu ufności 1-0.05 = 0.95
Var należy do przedziału [0.0098, 0.0501] przy założeniu poziomu ufności 1-0.02 = 0.98
Std należy do przedziału [0.1185, 0.2069] przy założeniu poziomu ufności 1-0.1 = 0.9


### Zadanie 2: Estymacja bootstrapowa wartości współczynnika korelacji liniowej [4 pkt]

Przygotuj funkcję `bootstrap_correlation_estimation()`, która korzystając z metody bootstrap będzie dokonywała estymacji przedziałowej wartości współczynnika korelacji liniowej Pearsona dla danych wejściowych w postaci szeregu szczegółowego zgodnie ze schematem z załączonego do notatnika zestawu wzorów.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`r należy do przedziału [X, Y] przy założeniu poziomu ufności 1-𝛼 = Z`

gdzie X, Y i Z są automatycznie uzupełniane przez funkcję, a X i Y dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

Uwagi do wykonania zadania:
 - Automatyczne testy zakładają losowanie prób wg następującego schematu - kolejne próby są losowane w pęli for, a funkcja losująca próbę (np. `pd.sample()`) w kolejnych losowaniach ma ustawioną wartość parametru odpowiadającego za ziarno generatora liczb losowych na wartość `random_state`, `random_state+1`, `random_state+2` itd. 
 - Celem zadania jest napisanie funkcji od podstaw, w rozwiązaniu nie należy korzystać z gotowych funkcji służących do estymacji bootstrapowej.

In [6]:
def bootstrap_correlation_estimation(data, number_of_samples, alpha, random_state):
    n = len(data)
    correlations = []
    for _ in range(number_of_samples):
        np.random.seed(random_state)
        random_state += 1
        bootstrap_sample = data.sample(n, replace=True).values
        correlation_matrix = np.corrcoef(bootstrap_sample, rowvar=False) # corrcoef,  macierz korelacji 
        correlation_coefficient = correlation_matrix[0, 1]
        correlations.append(correlation_coefficient)
    lower_bound = np.quantile(correlations, alpha/2)
    upper_bound = np.quantile(correlations, 1-alpha/2)
    print(f"r należy do przedziału [{lower_bound:.4f}, {upper_bound:.4f}] przy założeniu poziomu ufności 1-{alpha} = {1-alpha}")
    return lower_bound, upper_bound
    

In [7]:
# Komórka testowa
assert np.all(np.isclose(bootstrap_correlation_estimation(sample_2, 1200, 0.05, 54), (0.9292256084679922, 0.9816422271248535)))
assert np.all(np.isclose(bootstrap_correlation_estimation(sample_2, 1500, 0.02, 14), (0.9171033819425604, 0.9840550768251518)))

r należy do przedziału [0.9292, 0.9816] przy założeniu poziomu ufności 1-0.05 = 0.95
r należy do przedziału [0.9171, 0.9841] przy założeniu poziomu ufności 1-0.02 = 0.98


### Zadanie 3: Estymacja bootstrapowa współczynników równania regresji liniowej [6 pkt]

Przygotuj funkcję `bootstrap_linear_regression_coefficients_estimation()`, która korzystając z metody bootstrap będzie dokonywała estymacji przedziałowej wartości współczynników równania regresji liniowej dla danych wejściowych w postaci szeregu szczegółowego zgodnie ze schematem z załączonego do notatnika zestawu wzorów.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`a należy do przedziału [X1, Y1] przy założeniu poziomu ufności 1-𝛼 = Z1` </br>
`b należy do przedziału [X2, Y2] przy założeniu poziomu ufności 1-𝛼 = Z2`

gdzie X1, X2, Y1, Y2, Z1 i Z2 są automatycznie uzupełniane przez funkcję, a X1, X2, Y1 i Y2 dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

Uwagi do wykonania zadania:
 - Automatyczne testy zakładają losowanie prób wg następującego schematu - kolejne próby są losowane w pęli for, a funkcja losująca próbę (np. `pd.sample()`) w kolejnych losowaniach ma ustawioną wartość parametru odpowiadającego za ziarno generatora liczb losowych na wartość `random_state`, `random_state+1`, `random_state+2` itd. 
 - Celem zadania jest napisanie funkcji od podstaw, w rozwiązaniu nie należy korzystać z gotowych funkcji służących do estymacji bootstrapowej.

In [4]:
def bootstrap_linear_regression_coefficients_estimation(data, number_of_samples, alpha, random_state):
    n = len(data)
    coefficients = {'a': [], 'b': [] }
    for i in range(number_of_samples):
        np.random.seed(random_state)
        random_state += 1
        bootstrap_sample = data.sample(n=n, replace=True)
        x_mean = bootstrap_sample['X'].mean()
        y_mean = bootstrap_sample['Y'].mean()
        xy_mean = (bootstrap_sample['X'] * bootstrap_sample['Y']).mean()
        x_squared_mean = (bootstrap_sample['X'] ** 2).mean()
        a = (xy_mean - x_mean * y_mean) / (x_squared_mean - x_mean ** 2) # 
        b = y_mean - a * x_mean
        coefficients['a'].append(a)
        coefficients['b'].append(b)
    coefficients_df = pd.DataFrame(coefficients)
    confidence_intervals = coefficients_df.quantile([alpha / 2, 1 - alpha / 2]) # np.quantile(correlations, alpha/2)
    #print(confidence_intervals)
    a_lower, a_upper = confidence_intervals.loc[alpha / 2, 'a'], confidence_intervals.loc[1 - alpha / 2, 'a']
    b_lower, b_upper = confidence_intervals.loc[alpha / 2, 'b'], confidence_intervals.loc[1 - alpha / 2, 'b'] # 
    print(f"a należy do przedziału [{a_lower:.4f}, {a_upper:.4f}] przy założeniu poziomu ufności 1-{alpha:.2f}")
    print(f"b należy do przedziału [{b_lower:.4f}, {b_upper:.4f}] przy założeniu poziomu ufności 1-{alpha:.2f}")
    return ( (a_lower, a_upper), (b_lower, b_upper) )
    

# 

In [5]:
# Komórka testowa
assert np.all(np.isclose(bootstrap_linear_regression_coefficients_estimation(sample_2, 1800, 0.05, 71), ((1.8361393329086446, 2.2918023790827124), (-5.575502208944547, -3.6854174758043574))))
print()
assert np.all(np.isclose(bootstrap_linear_regression_coefficients_estimation(sample_2, 2100, 0.02, 62), ((1.7964929430496048, 2.337236979144576), (-5.771238137193083, -3.481203748551042))))

a należy do przedziału [1.8361, 2.2918] przy założeniu poziomu ufności 1-0.05
b należy do przedziału [-5.5755, -3.6854] przy założeniu poziomu ufności 1-0.05

a należy do przedziału [1.7965, 2.3372] przy założeniu poziomu ufności 1-0.02
b należy do przedziału [-5.7712, -3.4812] przy założeniu poziomu ufności 1-0.02


### Zadanie 4: Estymacja bootstrapowa za pomocą dedykowanych funkcji [7 pkt]

Korzystając z funkcji `scipy.stats.bootstrap()` dokonaj estymacji przedziałowej następujących parametrów na poziomie istotności 1-alpha = 0.98:
 - na podstawie zmiennej `sample_1`: średnia, wariancja, odchylenie standardowe,
 - na podstawie zmiannej `sample_2`: współczynnik korelacji liniowej Pearsona, współczynniki równania regresji liniowej a i b.
 
Wyniki zapisz do zmiennych o nazwach `mean_ci`, `var_ci`, `std_ci`, `r_ci`, `a_ci` oraz `b_ci` zawierających listy z dolnymi i górnymi granicami przedziałów ufności. Podczas obliczania wartości każdego z parametrów ustaw wartość argumentu funkcji `scipy.stats.bootstrap()` odpowiadającego za ziarno generatora liczb losowych na 25.

Uwagi do wykonania zadania:
 - Automatyczne testy przewidują obliczenie wartości wariancji i odchylenia standardowego dla wartości parametru `ddof=0`.

In [8]:
from scipy.stats import bootstrap, pearsonr, linregress
def mean_bootstrap(data):
    return np.mean(data)
def var_bootstrap(data):
    return np.var(data, ddof=0)
def std_bootstrap(data):
    return np.std(data, ddof=0)
random_seed = 25
dat = sample_1['X'].values
mean_ci = bootstrap((dat,), mean_bootstrap, random_state=random_seed, confidence_level=0.98) # random_state, confidence_level
mean_ci = (mean_ci.confidence_interval.low, mean_ci.confidence_interval.high)
print(mean_ci)


# super 

(4.890098551749027, 5.063500727815335)


In [9]:
var_ci = bootstrap((dat,), var_bootstrap, random_state=random_seed, confidence_level=0.98)
var_ci = (var_ci.confidence_interval.low, var_ci.confidence_interval.high)
print(var_ci)

(0.013001391902892368, 0.05670851692597062)


In [10]:
std_ci = bootstrap((dat,), std_bootstrap, random_state=random_seed, confidence_level=0.98)
std_ci = (std_ci.confidence_interval.low, std_ci.confidence_interval.high)
print(std_ci)

(0.11423521149909441, 0.23829111055240196)


In [14]:
# r
# YOUR CODE HERE
r_ci=bootstrap((sample_2["X"],sample_2["Y"]),sp.stats.pearsonr,confidence_level=0.98,random_state=25,paired = True).confidence_interval    # pearsonr, linregress
print(r_ci) #personr zwraca 2 rzeczy, interesuje nas 1, dlatego [0]
r_ci = (r_ci[0][0],r_ci[1][0])
r_ci

ConfidenceInterval(low=array([9.15671578e-01, 1.73537294e-18]), high=array([9.82274370e-01, 7.88152579e-11]))


(0.9156715777126913, 0.9822743695601275)

In [12]:
# a
# YOUR CODE HERE

a_ci=sp.stats.bootstrap((sample_2["X"],sample_2["Y"]),sp.stats.linregress,confidence_level=0.98,random_state=25,paired = True).confidence_interval
#linregress zwraca 5 rzeczy, interesuje nas pierwsza z nich
a_ci=(a_ci[0][0],a_ci[1][0])
a_ci 


##########################################################################################

# b
# YOUR CODE HERE
b_ci=sp.stats.bootstrap((sample_2["X"],sample_2["Y"]),sp.stats.linregress,confidence_level=0.98,random_state=25,paired = True).confidence_interval
print(b_ci)
#linregress zwraca 5 rzeczy, teraz interesuje nas 2 z nich
b_ci=(b_ci[0][1],b_ci[1][1])
b_ci

ConfidenceInterval(low=array([ 1.79294834e+00, -5.70093478e+00,  9.15671578e-01,  1.73537294e-18,
        8.52145410e-02]), high=array([ 2.32821116e+00, -3.42922833e+00,  9.82274370e-01,  7.88152579e-11,
        1.82704580e-01]))


(-5.700934784848322, -3.4292283266002834)

In [13]:
# Komórka testowa
assert np.all(np.isclose(mean_ci, (4.890098551749027, 5.063500727815335)))
assert np.all(np.isclose(var_ci, (0.013001391902892368, 0.05670851692597062)))
assert np.all(np.isclose(std_ci, (0.11423521149909441, 0.23829111055240196)))
assert np.all(np.isclose(r_ci, (0.9156715777126913, 0.9822743695601276)))
assert np.all(np.isclose(a_ci, (1.792948341896073, 2.3282111601384945)))
assert np.all(np.isclose(b_ci, (-5.700934784848322, -3.4292283266002834)))